In [1]:
# notebook that reads output_stage1 cleans data and writes to output_stage2 
%pylab inline 

import pandas as pd
import seaborn
import requests
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'svg'
seaborn.set_style("white")

Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_csv('output_stage1.csv',parse_dates=['DATETIME'])
df.drop('Unnamed: 0',axis =1, inplace = True)
df.head()


,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/28/2018,00:00:00,REGULAR,6598847,2235829,2018-04-28 00:00:00
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/28/2018,04:00:00,REGULAR,6598864,2235830,2018-04-28 04:00:00
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/28/2018,08:00:00,REGULAR,6598880,2235863,2018-04-28 08:00:00
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/28/2018,12:00:00,REGULAR,6598961,2235955,2018-04-28 12:00:00
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/28/2018,16:00:00,REGULAR,6599175,2236015,2018-04-28 16:00:00


In [3]:
df.shape

(2557801, 12)

In [4]:
df.DATETIME.head()

0   2018-04-28 00:00:00
1   2018-04-28 04:00:00
2   2018-04-28 08:00:00
3   2018-04-28 12:00:00
4   2018-04-28 16:00:00
Name: DATETIME, dtype: datetime64[ns]

In [5]:
turnstile =['STATION', 'UNIT', 'C/A', 'SCP']

df['ENTRY_DIFF'] = (df.groupby(turnstile)['ENTRIES']
                           .diff()) 

In [6]:
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME,ENTRY_DIFF
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/28/2018,00:00:00,REGULAR,6598847,2235829,2018-04-28 00:00:00,NaN
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/28/2018,04:00:00,REGULAR,6598864,2235830,2018-04-28 04:00:00,17.0
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/28/2018,08:00:00,REGULAR,6598880,2235863,2018-04-28 08:00:00,16.0
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/28/2018,12:00:00,REGULAR,6598961,2235955,2018-04-28 12:00:00,81.0
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/28/2018,16:00:00,REGULAR,6599175,2236015,2018-04-28 16:00:00,214.0


In [7]:
df['EXIT_DIFF'] = (df.groupby(turnstile)['EXITS']
                           .diff()) 
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME,ENTRY_DIFF,EXIT_DIFF
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/28/2018,00:00:00,REGULAR,6598847,2235829,2018-04-28 00:00:00,NaN,NaN
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/28/2018,04:00:00,REGULAR,6598864,2235830,2018-04-28 04:00:00,17.0,1.0
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/28/2018,08:00:00,REGULAR,6598880,2235863,2018-04-28 08:00:00,16.0,33.0
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/28/2018,12:00:00,REGULAR,6598961,2235955,2018-04-28 12:00:00,81.0,92.0
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/28/2018,16:00:00,REGULAR,6599175,2236015,2018-04-28 16:00:00,214.0,60.0


In [8]:
bignum = 100_000

df = df[
    (df["EXIT_DIFF"] > 0) &
    (df["ENTRY_DIFF"] > 0) &
    (df["EXIT_DIFF"] < bignum) &
    (df["ENTRY_DIFF"] < bignum)
]

In [9]:
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME,ENTRY_DIFF,EXIT_DIFF
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/28/2018,04:00:00,REGULAR,6598864,2235830,2018-04-28 04:00:00,17.0,1.0
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/28/2018,08:00:00,REGULAR,6598880,2235863,2018-04-28 08:00:00,16.0,33.0
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/28/2018,12:00:00,REGULAR,6598961,2235955,2018-04-28 12:00:00,81.0,92.0
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/28/2018,16:00:00,REGULAR,6599175,2236015,2018-04-28 16:00:00,214.0,60.0
5,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/28/2018,20:00:00,REGULAR,6599456,2236074,2018-04-28 20:00:00,281.0,59.0


In [10]:
df.shape

(2080857, 14)

In [11]:
# df.DESC.value_counts()

In [12]:
# df.loc[df['DESC']=='REGULAR','TIME'].value_counts()

In [13]:
df.to_csv('output_stage2.csv')